In [2]:
"""Train the model"""

import argparse
import logging
import os
import random

import tensorflow as tf

from read_tfrecords import read_tfrecords
from model.input_fn import input_fn
from model.utils import Params
from model.utils import set_logger
from model.utils import save_dict_to_json
from model.model_fn import model_fn
from model.training import train_and_evaluate

In [3]:
model_dir = 'experiments/base_model/'
data_dir_1 = 'data/SIGNS_TFRecord/'
data_dir_2 = 'data/SIGNS_TFRecord_v2/'

In [4]:
# Set the random seed for the whole graph for being able to reproduce the experiments
tf.set_random_seed(230)

# Load the parameters from json file

json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = Params(json_path)

# Check that we are not overwriting some previous experiment
# Comment these lines if you are developing your model and don't care about overwritting
# model_dir_has_best_weights = os.path.isdir(os.path.join(args.model_dir, "best_weights"))
# overwritting = model_dir_has_best_weights and args.restore_from is None
# assert not overwritting, "Weights found in model_dir, aborting to avoid overwrite"

# Set the logger
set_logger(os.path.join(model_dir, 'train.log'))

In [5]:
# Read the number of examples of each dataset (This could be optimized by storing the number in an external file
# when creating the TFRecords)
params.train_size =  len([x for x in tf.python_io.tf_record_iterator(os.path.join(data_dir_2, 'train.tfrecords'))])
logging.info('Train size: {}'.format(params.train_size))

Train size: 864


In [6]:

train_inputs_1 = input_fn(
    tfrecord_filenames=os.path.join(data_dir_1, 'train.tfrecords'),
    num_records=params.train_size,
    params=params
)

train_inputs_2 = input_fn(
    tfrecord_filenames=os.path.join(data_dir_2, 'train.tfrecords'),
    num_records=params.train_size,
    params=params
)

Tensor("DecodeRaw:0", shape=(?,), dtype=float32)
Tensor("Reshape:0", shape=(64, 64, 3), dtype=float32)
Tensor("DecodeRaw:0", shape=(?,), dtype=float32)
Tensor("Reshape:0", shape=(64, 64, 3), dtype=float32)


In [7]:
with tf.Session() as sess:
    
    # Initialize all global and local variables
    sess.run(train_inputs_1['it_init_op'])
    sess.run(train_inputs_2['it_init_op'])
    
    #img1, lbl1 = sess.run([train_inputs_1['images'], train_inputs_1['labels']])
    img2, lbl2 = sess.run([train_inputs_2['images'], train_inputs_2['labels']])

InvalidArgumentError: Input to reshape is a tensor with 3072 values, but the requested shape has 12288
	 [[Node: Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32](DecodeRaw, Reshape/shape)]]
	 [[Node: IteratorGetNext_1 = IteratorGetNext[output_shapes=[[?,64,64,3], [?]], output_types=[DT_FLOAT, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator_1)]]

In [1]:
print(lbl1.shape)

NameError: name 'lbl1' is not defined

In [ ]:
# Define the model
train_model_spec_2 = model_fn('train', train_inputs_2, params, reuse = False)

In [ ]:
with tf.Session() as sess:
    
    sess.run(train_model_spec_2['it_init_op'])
    
    img = sess.run(train_model_spec_2['images'])
    lbl = sess.run(train_model_spec_2['labels'])


In [ ]:
lbl.shape

In [ ]:
with tf.Session() as sess:
    
    # Initialize all global and local variables
    sess.run(train_inputs_1['it_init_op'])
    sess.run(train_inputs_2['it_init_op'])
    
    img1, lbl1 = sess.run([train_inputs_1['images'], train_inputs_1['labels']])
    img2, lbl2 = sess.run([train_inputs_1['images'], train_inputs_1['labels']])

In [ ]:
print(lbl1.shape)
print(img1.shape)

In [ ]:
print(lbl2.shape)
print(img2.shape)